# Case FortBrasil
***

###  **Legenda:**
- **ID_CONTA:** Identificador da conta;
- **DT_VENCIMENTO:** Data vencimento da fatura;
- **DS_ROLAGEM:** Indicador que define se o cliente pagou a fatura do mês anterior ou não. Se o valor for *FX0* quer dizer que o cliente pagou a fatura do mês anterior e se o valor for *FX1* quer dizer que o cliente *não* pagou a fatura do mês anterior;
- **VL_FATURA:** Valor da Fatura atual.

## Questão 01
### 1.1. Qual o percentual de faturas emitidas por mês no qual os clientes não pagaram a fatura anterior? 
***

### Importação das bibliotecas

Seleção de bibliotecas para manipulação de dados, criação de gráficos e tradução dos meses a serem usados do inglês para o português

In [45]:
import pandas as pd
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None
import locale

In [44]:
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR')
except:
    locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil')

### Leitura da base de dados

In [29]:
df_inicial = pd.read_csv('Questão 1 - Base.txt', sep = "\t")
df_inicial.head()

,ID_CONTA,DT_VENCIMENTO,DS_ROLAGEM,VL_FATURA
0,402467,2019-01-01,FX0,210.00
1,403001,2019-01-01,FX1,1097.18
2,403499,2019-01-01,FX0,1209.88
3,403670,2019-01-01,FX0,667.40
4,404183,2019-01-01,FX1,1239.52


### Verificação de NaN

É sempre interessante que tratemos os NaN, pois eles interferem em nossa modelagem de dados. Para esse caso, não foi encontrado nenhum.

In [46]:
# Sem registros de NaN nas colunas
df_inicial.isna().sum()

ID_CONTA         0
DT_VENCIMENTO    0
DS_ROLAGEM       0
VL_FATURA        0
dtype: int64

## Solução
Qual o percentual de faturas emitidas por mês no qual os clientes não pagaram a fatura anterior?

Para essa questão devemos seguir os seguintes passos:

1. Filtrar principais colunas a serem usadas (isso nos permite trabalhar com dados, de fato, essenciais para a modelagem)
2. Mudar tipo de dado da tabela "DT_VENCIMENTO" para datetime, afim de conseguirmos filtrar os dados por mês
3. Criar uma coluna contador para 
3. Somar a quantidade de faturas não pagas de cada mês
4. Calcular percentual desejado  
O primeiro passo é observar que tipo


### 1. Filtrar colunas importantes a serem usadas 

Após o tratamento dos dados, devemos selecionar a(s) informação(ões) útil(eis) para a manipulação dos dados. Assim, de acordo com o enunciado, precisamos apenas do mês da fatura emitida e do código dos clientes que não pagaram a fatura anterior.

In [36]:
df_filtrado = df_inicial[['DT_VENCIMENTO','DS_ROLAGEM']]
df_filtrado.head()

,DT_VENCIMENTO,DS_ROLAGEM
0,2019-01-01,FX0
1,2019-01-01,FX1
2,2019-01-01,FX0
3,2019-01-01,FX0
4,2019-01-01,FX1


### 2. Transformação 'DT_VENCIMENTO' de object para datetime
Primeiro devemos verificar se o tipo de dado da coluna 'DT_VENCIMENTO' já é datetime ou é outro

In [47]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2728249 entries, 0 to 2728248
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   DT_VENCIMENTO  object
 1   DS_ROLAGEM     object
 2   CONTAGEM       int64 
dtypes: int64(1), object(2)
memory usage: 62.4+ MB


Confirmado que a coluna 'DT_VENCIMENTO' é um object, devemos realizar sua mudança de tipo para datetime, com o intuito de conseguirmos filtrar os meses de faturamento para o cálculo percentual

In [38]:
df_filtrado['DT_VENCIMENTO'] = pd.to_datetime(df_filtrado['DT_VENCIMENTO'])
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2728249 entries, 0 to 2728248
Data columns (total 2 columns):
 #   Column         Dtype         
---  ------         -----         
 0   DT_VENCIMENTO  datetime64[ns]
 1   DS_ROLAGEM     object        
dtypes: datetime64[ns](1), object(1)
memory usage: 41.6+ MB


In [39]:
df_filtrado['DT_VENCIMENTO'] = df_filtrado['DT_VENCIMENTO'].apply(lambda x: x.strftime("%B - %Y"))
df_filtrado.head()

,DT_VENCIMENTO,DS_ROLAGEM
0,janeiro - 2019,FX0
1,janeiro - 2019,FX1
2,janeiro - 2019,FX0
3,janeiro - 2019,FX0
4,janeiro - 2019,FX1


### 3. Criar coluna para contagem

In [41]:
df_filtrado['CONTAGEM'] = 1
df_filtrado.head()

,DT_VENCIMENTO,DS_ROLAGEM,CONTAGEM
0,janeiro - 2019,FX0,1
1,janeiro - 2019,FX1,1
2,janeiro - 2019,FX0,1
3,janeiro - 2019,FX0,1
4,janeiro - 2019,FX1,1


### 4. Criar um novo DataFrame com a soma das faturas pagas e não pagas anteriormente
-

In [42]:
df_agrupamento = df_filtrado.groupby(['DT_VENCIMENTO','DS_ROLAGEM'])['CONTAGEM'].sum()
df_agrupamento

DT_VENCIMENTO     DS_ROLAGEM
abril - 2019      FX0           276950
                  FX1            25380
agosto - 2019     FX0           288331
                  FX1            28736
fevereiro - 2019  FX0           285921
                  FX1            27928
janeiro - 2019    FX0           290548
                  FX1            22835
julho - 2019      FX0           281005
                  FX1            29889
junho - 2019      FX0           278389
                  FX1            25977
maio - 2019       FX0           271544
                  FX1            30321
março - 2019      FX0           273886
                  FX1            33432
setembro - 2019   FX0           235222
                  FX1            21955
Name: CONTAGEM, dtype: int64

### Porcentagem:

In [69]:
df_taxa = pd.DataFrame()
df_taxa['TAXA DE FX1 (%)'] = round((dfcase[dfcase['DS_ROLAGEM']=='FX1'].groupby('MES')['DS_ROLAGEM'].count()/dfcase.groupby('MES')['DS_ROLAGEM'].count())*100,2)
df_taxa.reset_index(inplace=True)
df_taxa['MES'] = ['janeiro','fevereiro','mar','abril','maio','junho','julho','agosto','setembro']
df_taxa

NameError: name 'dfcase' is not defined

In [67]:
df_faturas_emitidas_por_mes = pd.DataFrame()
df_faturas_emitidas_por_mes['Percentual FX1'] = df_agrupamento[df_agrupamento['DS_ROLAGEM'] == 'FX1'].groupby('DT_VENCIMENTO')['CONTAGEM'].count()
#/df_agrupamento.groupby('DT_VENCIMENTO')['CONTAGEM'].count(),4
#df_faturas_emitidas_por_mes['282 - 382'] = round((df_filtrado[(df_filtrado['RESPOSTA'] == 1) & (df_filtrado['INTERVALOS_DEFINIDOS'] == '282 - 382')].groupby('DT_ACORDO')['RESPOSTA'].count())/df_filtrado.groupby('DT_ACORDO')['RESPOSTA'].count(),4)*100
#df_faturas_emitidas_por_mes['383 - 483'] = round((df_filtrado[(df_filtrado['RESPOSTA'] == 1) & (df_filtrado['INTERVALOS_DEFINIDOS'] == '383 - 483')].groupby('DT_ACORDO')['RESPOSTA'].count())/df_filtrado.groupby('DT_ACORDO')['RESPOSTA'].count(),4)*100
#df_faturas_emitidas_por_mes['Maior que 484'] = round((df_filtrado[(df_filtrado['RESPOSTA'] == 1) & (df_filtrado['INTERVALOS_DEFINIDOS'] == 'Maior que 484')].groupby('DT_ACORDO')['RESPOSTA'].count())/df_filtrado.groupby('DT_ACORDO')['RESPOSTA'].count(),4)*100
df_faturas_emitidas_por_mes

KeyError: 'CONTAGEM'

In [118]:
#df_taxa['TAXA DE FX1 (%)'] = round((dfcase[dfcase['DS_ROLAGEM']=='FX1'].groupby('MES')['DS_ROLAGEM'].count()/dfcase.groupby('MES')['DS_ROLAGEM'].count())*100
#df_porcentagem['(%) Faturas não pagas'] = df_agrupamento[df_agrupamento['DS_ROLAGEM'] == 'FX1'].groupy('DT_VENCIMENTO')['DS_ROLAGEM'].count()

### Graficamente:

In [ ]:
# Gráfico

### 1.2. Tendo como referência todos os clientes que tiveram fatura emitida no mês de setembro, gere uma base para esses clientes com os seguintes calculados:
- Total de faturas emitidas nos últimos 6 meses (sem contar com a fatura de setembro);
- O valor médio de fatura nos últimos 6 meses (sem contar com a fatura de setembro);
- Quantidade de vezes que ele ficou sem pagar a fatura anterior nos últimos 6 meses (sem contar com a fatura de setembro). 
***